<a href="https://colab.research.google.com/github/fecsaba/FluxKiller/blob/main/FluxKiller_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Flux killer Load**
### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.

#### Crystools

### EasyUse
A ComfyUI használhatóságának javítása érdekében számos gyakran használt csomóponthoz optimalizáltak és integráltak.
### ITools
Az iTools néhány életminőség-csomópont, mint például a kép létrehozásához használt lehetséges prompt beolvasása, a prompt fájlba mentése új sorként, a promptok beolvasása többsoros fájlból.

### ComfyUI-Detail-Daemon
A Muerrilla sd-webui-Detail-Daemon portja a ComfyUI csomópontjaként, a szigmák beállításához, amelyek általában javítják a részleteket, és esetleg eltávolítják a nem kívánt bokeh- vagy háttérelmosódást, különösen a Flux modelleknél (de működik SDXL, SD1.5 és valószínűleg más modellek). Ha az értékeket túl messzire viszi, az túléles és/vagy HDR hatást eredményez. Itt négy csomópont található. A Multiply Sigma és a Lying Sigma Sampler szintén szerepelnek a részletek általános javításának alternatív módszereiként.

Detail Daemon Sampler
Detail Daemon Graph Sigmas (a szigmák beállításának vizuális ábrázolásához)
Szigmák szorzása (hontalan)
Hazug Sigma Sampler
Vegye figyelembe, hogy a Detail Daemon és a Lying Sigma Sampler csomópontok alapértelmezés szerint olyan egyéni mintavevő csomópontokkal működnek, mint például a SamplerCustomAdvanced. Ha nem egyéni mintavevő csomópontokkal szeretné használni őket, például KSamplera vagy KSamplerAdvanced, akkor létre kell hoznia egy egyéni mintavevő-előbeállítást a BlehSetSamplerPresetcsomópont segítségével, így kiválaszthatja az előre beállított értéket a mintavevő csomópontban található listából, az itt leírtak szerint .

### Rgthree Comfy
Csomópontok: Seed, Reroute, Context, Lora Loader Stack, Context Switch, Fast Muter. Ezek az egyéni csomópontok segítenek megszervezni az összetett munkafolyamatok felépítését.

#### ComfyUI's ControlNet Auxiliary Preprocessors
Plug-and-play ComfyUI csomópontkészletek ControlNet tippképek készítéséhez
"anime stílus, tiltakozás az utcán, cyberpunk város, egy rózsaszín hajú, arany szemű nő (nézi a nézőt) egy táblát tart a "ComfyUI ControlNet Aux" felirattal félkövér, neonrózsaszínnel a Flux.1 Dev-n
A kód másolás-beillesztése a https://github.com/lllyasviel/ControlNet/tree/main/annotator megfelelő mappáiból, és csatlakoztatva van a 🤗 Hubhoz .

<!-- #### Was node suite
Csomópontcsomag a ComfyUI-hoz számos új csomóponttal, például képfeldolgozással, szövegfeldolgozással és egyebekkel.


#### ComfyUI ControlAltAI Nodes
Fluxus mintavevő
A Flux Sampler csomópont egyetlen, áramvonalas csomópontban egyesíti a CustomSamplerAdvance csomópont és a bemeneti csomópontok funkcióit.

CFG beállítás: A CFG fix értéke 1.
Kondicionálás bemenet: Csak pozitív kondicionálás támogatott.
Kompatibilitás: Csak a Flux modellel kompatibilis mintavevőket és ütemezőket tartalmazza.
Látens kompatibilitás: Csak SD3 üres látens képet használjon. A normál üres látens képcsomópont nem kompatibilis.

## Florence2 in ComfyUI
A Florence-2 egy fejlett látásalap-modell, amely azonnali alapú megközelítést alkalmaz a látás és a látásnyelvi feladatok széles skálájának kezelésére. A Florence-2 képes értelmezni az egyszerű szöveges felszólításokat olyan feladatok végrehajtásához, mint a feliratozás, az objektumészlelés és a szegmentálás. Használja ki FLD-5B adatkészletünket, amely 5,4 milliárd megjegyzést tartalmaz 126 millió képen, hogy elsajátítsa a többfeladatos tanulást. A modell sorozatról-szekvenciára architektúrája lehetővé teszi, hogy mind a nulla lövés, mind a finomhangolt beállításokban kiváló legyen, így versenyképes látásalapmodellnek bizonyul. -->



# Set environment

In [3]:
from pathlib import Path
import os
import hashlib

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

OPTIONS.update({
    'USE_GOOGLE_DRIVE': USE_GOOGLE_DRIVE,
    'UPDATE_COMFY_UI': UPDATE_COMFY_UI,
    'USE_COMFYUI_MANAGER': USE_COMFYUI_MANAGER,
    'INSTALL_GGUF_NODE': INSTALL_GGUF_NODE,
    'INSTALL_CUSTOM_NODES_DEPENDENCIES': INSTALL_CUSTOM_NODES_DEPENDENCIES
})


print(f"Google Drive használata: {OPTIONS['USE_GOOGLE_DRIVE']}")
print(f"ComfyUI frissítése: {OPTIONS['UPDATE_COMFY_UI']}")
print(f"ComfyUI Manager használata: {OPTIONS['USE_COMFYUI_MANAGER']}")
print(f"GGUF Node telepítése: {OPTIONS['INSTALL_GGUF_NODE']}")
print(f"Custom Node dependenciák telepítése: {OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']}")

# Alapértelmezett munkakönyvtár Colab alatt
WORKSPACE_BASE = "/content"
WORKSPACE = f"{WORKSPACE_BASE}/ComfyUI"
TEMP_MODELS_DIR = f"{WORKSPACE_BASE}/temp_models/"

# Google Drive használata esetén
if OPTIONS['USE_GOOGLE_DRIVE']:
    print("Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE_BASE = "/content/drive/MyDrive"
    WORKSPACE = f"{WORKSPACE_BASE}/ComfyUI"
    print(f"Google Drive útvonal használata: {WORKSPACE}")

comfyui_path = WORKSPACE  # Egyértelműsítés

os.environ['COMFYUI_PATH'] = comfyui_path
os.environ['WORKSPACE'] = WORKSPACE

Google Drive használata: True
ComfyUI frissítése: False
ComfyUI Manager használata: True
GGUF Node telepítése: True
Custom Node dependenciák telepítése: True
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive útvonal használata: /content/drive/MyDrive/ComfyUI


# New version for init

In [4]:
from pathlib import Path
import os
import hashlib

if 'WORKSPACE' in vars():
  print("A 'WORKSPACE' nevű változó létezik.")
else:
  print("A 'WORKSPACE' nevű változó nem létezik.")
  exit(1)

if not os.path.exists(WORKSPACE):
    print("-= Initial setup ComfyUI =-")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}

!chmod -R 755 $WORKSPACE

%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  print("-= Updating ComfyUI =-")
  !chmod 755 .ci/**/*.bat .ci/**/*.py .ci/**/*.txt
  !git pull

print("-= Install dependencies =-")
!pip3 install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  !chmod 755 ComfyUI-Manager/**/*.sh
  if not os.path.exists("ComfyUI-Manager"):
      print("-= Initial setup ComfyUI-Manager =-")
      !git clone https://github.com/ltdrdata/ComfyUI-Manager
  else:
    %cd ComfyUI-Manager
    if os.path.exists(".git"):
        !git fetch --all
        !git reset --hard origin/main
    else:
        print("ComfyUI-Manager repository is invalid. Reinstalling...")
        !rm -rf ComfyUI-Manager
        !git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd $WORKSPACE/custom_nodes

if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE/custom_nodes
  if not os.path.exists("ComfyUI-GGUF"):
    !git clone https://github.com/city96/ComfyUI-GGUF
    !pip install -r ComfyUI-GGUF/requirements.txt
  else:
      print("GGUF node már telepítve")


%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  print("-= Install custom nodes dependencies =-")
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

print("-= Install nodes =-")
print("-= **************** =-")
print("-= Install Crystools =-")
%cd $WORKSPACE/custom_nodes
if not os.path.exists("ComfyUI-Crystools"):
  !git clone https://github.com/crystian/ComfyUI-Crystools.git
  if os.path.exists("ComfyUI-Crystools/requirements.txt"):
      !pip install -r ComfyUI-Crystools/requirements.txt
  else:
      print("Nincs requirements.txt a ComfyUI-Crystools mappában.")
else:
    print("Crystools már telepítve")

print("-= Install EasyUse =-")
%cd $WORKSPACE/custom_nodes
if not os.path.exists("ComfyUI-Easy-Use"):
    !git clone https://github.com/yolain/ComfyUI-Easy-Use.git
    !pip install -r ComfyUI-Easy-Use/requirements.txt
else:
    print("EasyUse már telepítve")

def install_custom_node(comfyui_path, repo_url, node_folder_name, skip_install=False):
    custom_nodes_path = os.path.join(comfyui_path, "custom_nodes")

    if not os.path.exists(custom_nodes_path):
        os.makedirs(custom_nodes_path)
    print(f"custom_nodes mappa elérési útvonala: {custom_nodes_path}")

    destination_path = os.path.join(custom_nodes_path, node_folder_name)

    if not os.path.exists(destination_path):
        !git clone {repo_url} {destination_path}
        print(f"{node_folder_name} telepítése kész.")
    else:
        print(f"{node_folder_name} már telepítve van.")
        return # Itt befejezi, ha már telepítve van

    requirements_path = os.path.join(destination_path, "requirements.txt")
    if os.path.exists(requirements_path) and not skip_install:
        print(f"Dependenciák telepítése: {node_folder_name}")
        !pip install -r {requirements_path}
    elif not skip_install:
       print(f"Nincs requirements.txt fájl a {node_folder_name}-ben, nem szükséges dependenciákat telepíteni.")
    else:
        print(f"Dependenciák telepítése a {node_folder_name}-hez skipelve.")

comfyui_path = "/content/drive/MyDrive/ComfyUI"


repo_url = "https://github.com/Fannovel16/comfyui_controlnet_aux.git"
node_folder_name = "comfyui_controlnet_aux"

install_custom_node(comfyui_path, repo_url, node_folder_name)


repo_url = "https://github.com/rgthree/rgthree-comfy.git"
node_folder_name = "rgthree-comfy"

install_custom_node(comfyui_path, repo_url, node_folder_name)

repo_url = "https://github.com/Jonseed/ComfyUI-Detail-Daemon.git"
node_folder_name = "ComfyUI-Detail-Daemon"

install_custom_node(comfyui_path, repo_url, node_folder_name)


%cd $WORKSPACE
print("Telepítés befejezve. Indítsd újra a ComfyUI-t!")

A 'WORKSPACE' nevű változó létezik.
/content/drive/MyDrive/ComfyUI
-= Install dependencies =-
Looking in indexes: https://download.pytorch.org/whl/cu121
/content/drive/MyDrive/ComfyUI/custom_nodes
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Fetching origin
fatal: bad object refs/heads/main
r
error: https://github.com/ltdrdata/ComfyUI-Manager did not send all necessary objects

error: Could not fetch origin
Updating files: 100% (84/84), done.
HEAD is now at 3d80ed95 update DB
/content/drive/MyDrive/ComfyUI/custom_nodes
/content/drive/MyDrive/ComfyUI/custom_nodes
GGUF node már telepítve
/content/drive/MyDrive/ComfyUI
-= Install custom nodes dependencies =-
----------------------------------------------------------------------------------------------------
Restoring [1/7]: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Install: pip packages

## ComfyUI-Manager: EXECUTE => ['/usr/bin/python3', '-m', 'pip', 'install', 'GitPython']

## ComfyUI-Manager: EXECUTE =>

# Load models

In [6]:
import os
import subprocess
import hashlib
import shlex
import time
import shutil

def is_valid_file(filepath, expected_hash=None):
    """Ellenőrzi a fájlt (pl. hash alapján).
       Ha expected_hash None, akkor az ellenőrzést kihagyja."""
    if not os.path.isfile(filepath):
        print(f"Figyelmeztetés: A '{filepath}' nem fájl.")
        return False
    if expected_hash is None:
        print(f"Tájékoztatás: A '{filepath}' fájlt átugorjuk, mivel nincs hash ellenőrzés.")
        return True
    try:
        sha256_hash = hashlib.sha256()
        with open(filepath, "rb") as f:
            while True:
                chunk = f.read(4096)
                if not chunk:
                    break
                sha256_hash.update(chunk)
        if sha256_hash.hexdigest() == expected_hash:
            print(f"Tájékoztatás: A '{filepath}' fájl hash értéke megegyezik a várttal.")
            return True
        else:
            print(f"Figyelmeztetés: A '{filepath}' fájl hash értéke nem egyezik meg a várttal.")
            return False
    except Exception as e:
        print(f"Hiba a hash ellenőrzéskor: {e}")
        return False

def create_symbolic_links(source_dir, file_list):
    """Létrehozza a szimbolikus linkeket a fájlokhoz a file_list alapján.

       source_dir: A forráskönyvtár útvonala.
       file_list: A letöltendő fájlok listája, a szimbolikus linkek helyeivel.
    """
    for file_info in file_list:
      filename = file_info["filename"]
      source_path = os.path.join(source_dir, filename)
      if "link_target_directories" in file_info: #Csak akkor ha van link_target_directories
        target_dirs = file_info["link_target_directories"]
        for target_dir in target_dirs:

            target_path = os.path.join(WORKSPACE, "models", target_dir, filename)
            try:
                os.makedirs(target_dir, exist_ok=True)  # Célkönyvtárak létrehozása (ha nem létezik)
                os.symlink(source_path, target_path)  # Szimbolikus link létrehozása
                print(f"Szimbolikus link létrehozva: {source_path} -> {target_path}")
            except FileExistsError:
                print(f"A célfájl már létezik: {target_path}. A szimbolikus linket nem hoztuk létre")
            except Exception as e:
                print(f"Hiba a link létrehozásakor: {e}")

def download_file(url, target_dir, filename, wget_params=None, expected_hash=None, downloaded_files=None):
    """Letölt egy fájlt a megadott URL-ről.
        url: A letöltendő fájl URL-je.
        target_dir: A célkönyvtár útvonala.
        filename: A letöltött fájl neve.
        wget_params: Opcionális wget paraméterek stringként.
        expected_hash: Opcionális hash ellenőrzés
        downloaded_files: A letöltött fájlok listája
        """
    model_path = os.path.join(WORKSPACE, "models")
    target_path = os.path.join(model_path, target_dir, filename)
    os.makedirs(target_dir, exist_ok=True)

    if downloaded_files and target_path in downloaded_files:
        print(f"Tájékoztatás: A '{target_path}' fájl már létezik, ellenőrzése folyamatban.")
        if is_valid_file(target_path, expected_hash):
            print(f"Tájékoztatás: A '{target_path}' fájl érvényes, a letöltés átugorva.")
            return True
        else:
            print(f"Figyelmeztetés: A '{target_path}' fájl érvénytelen. A fájl újratöltése következik.")

    if wget_params is None:
        wget_params = ""


    wget_command_parts = ["wget", "-c", url, wget_params, "-O", target_path]
    wget_command = shlex.join(wget_command_parts)
    print(f"Tájékoztatás: Fájl letöltése: {url} -> {target_path}")
    try:
      start_time = time.time()
      result = subprocess.run(wget_command, shell=True, capture_output=True, check=False)
      end_time = time.time()

      if result.returncode != 0:
          print(f"Hiba a fájl letöltésekor: {result.stderr.decode()}")
          return False
      if is_valid_file(target_path, expected_hash):
            print(f"Tájékoztatás: Fájl sikeresen letöltve és ellenőrizve: {target_path} (idő: {end_time-start_time:.2f}s)")
            if downloaded_files is not None:
                downloaded_files[target_path] = True
            return True
      else:
            print(f"Figyelmeztetés: A fájl '{target_path}' érvénytelen, a letöltés sikeres, de az ellenőrzés nem sikerült. (idő: {end_time-start_time:.2f}s)")
            return False
    except subprocess.CalledProcessError as e:
        print(f"Hiba a fájl letöltésekor: {e}")
        return False

if __name__ == "__main__":
    if 'WORKSPACE' in vars():
        print("A 'valtozo' nevű változó létezik.")
    else:
        print("A 'valtozo' nevű változó nem létezik.")
        exit(1)

    download_directory = "/content/downloaded_files"  # A letöltési célkönyvtár
    file_list = [
            # Download the RealVisXL model from Hugging Face and save it in the ./models/checpoints
           {
               "url": "https://huggingface.co/SG161222/RealVisXL_V5.0_Lightning/resolve/main/RealVisXL_V5.0_Lightning_fp16.safetensors",
                "filename": "RealVisXL_V5.0_Lightning_fp16.safetensors",
                "expected_hash": None,
                "wget_params": "--no-check-certificate",
                "link_target_directories": ["checkpoints"]
            },
            # sdxl_vae.safetensors save vae
            {
            "url": "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",
             "filename": "sdxl_vae.safetensors",
            "expected_hash": None,
             "wget_params": "--no-check-certificate",
             "link_target_directories": ["vae"]
            },
            # Download the FLUX model from Hugging Face and save it in the ./models/unet directory
            {
                "url": "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf",
                "filename": "flux1-dev-Q4_K_S.gguf",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["unet"]
            },
            # Download the CLIP T5 model from Hugging Face and save it in ./models/clip
            {
                "url": "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf",
                "filename": "t5-v1_1-xxl-encoder-Q4_K_S.gguf",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["clip"]
            },
            # Download the safetensors file for the CLIP model
            {
                "url": "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors",
                "filename": "clip_l.safetensors",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["clip"]
            },
            # Download the VAE model from Hugging Face and save it in ./models/vae
            {
                "url": "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors",
                "filename": "ae.safetensors",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["vae"]
            },
            # # Download the ControlNet models save models/controlnet
            # If you want to use a specific ControlNet model, remove the hashtag (#) from the corresponding line.
            {
                "url": "https://huggingface.co/XLabs-AI/flux-controlnet-depth-v3/resolve/main/flux-depth-controlnet-v3.safetensors",
                "filename": "flux-depth-controlnet-v3.safetensors",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["controlnet"]
            },
            {
                "url": "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-canny-controlnet-v3.safetensors",
                "filename": "flux-canny-controlnet-v3.safetensors",
                 "wget_params": "--no-check-certificate",
                "link_target_directories": ["controlnet"]
            },
            {
                "url": "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-hed-controlnet-v3.safetensors",
                "filename": "flux-hed-controlnet-v3.safetensors",
                 "wget_params": "--no-check-certificate",
                 "link_target_directories": ["controlnet"]
            },
        ]


    downloaded_files = {}
    for file_info in file_list:
        url = file_info["url"]
        filename = file_info["filename"]
        wget_params = file_info.get("wget_params")
        expected_hash = file_info.get("expected_hash")
        download_file(url, download_directory, filename, wget_params, expected_hash, downloaded_files)

    # Szimbolikus link létrehozása a letöltött fájlokhoz
    create_symbolic_links(download_directory, file_list)


    # A teszt mappa eltávolítása
    #shutil.rmtree(download_directory)

A 'valtozo' nevű változó létezik.
Tájékoztatás: Fájl letöltése: https://huggingface.co/SG161222/RealVisXL_V5.0_Lightning/resolve/main/RealVisXL_V5.0_Lightning_fp16.safetensors -> /content/downloaded_files/RealVisXL_V5.0_Lightning_fp16.safetensors
Tájékoztatás: A '/content/downloaded_files/RealVisXL_V5.0_Lightning_fp16.safetensors' fájlt átugorjuk, mivel nincs hash ellenőrzés.
Tájékoztatás: Fájl sikeresen letöltve és ellenőrizve: /content/downloaded_files/RealVisXL_V5.0_Lightning_fp16.safetensors (idő: 0.19s)
Tájékoztatás: Fájl letöltése: https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors -> /content/downloaded_files/sdxl_vae.safetensors
Tájékoztatás: A '/content/downloaded_files/sdxl_vae.safetensors' fájlt átugorjuk, mivel nincs hash ellenőrzés.
Tájékoztatás: Fájl sikeresen letöltve és ellenőrizve: /content/downloaded_files/sdxl_vae.safetensors (idő: 0.29s)
Tájékoztatás: Fájl letöltése: https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K

### Run ComfyUI

In [ ]:
if 'WORKSPACE' in vars():
  print("A 'valtozo' nevű változó létezik.")
else:
  print("A 'valtozo' nevű változó nem létezik.")
  exit(1)

!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

# Szöveg színei:

# \033[30m - Fekete

# \033[31m - Piros

# \033[32m - Zöld

# \033[33m - Sárga

# \033[34m - Kék

# \033[35m - Lila (Magenta)

# \033[36m - Cián

# \033[37m - Világos Szürke

# \033[90m - Sötét Szürke

# \033[91m - Világos Piros

# \033[92m - Világos Zöld

# \033[93m - Világos Sárga

# \033[94m - Világos Kék

# \033[95m - Világos Lila (Magenta)

# \033[96m - Világos Cián

# \033[97m - Fehér

# Háttér színei:

# \033[40m - Fekete háttér

# \033[41m - Piros háttér

# \033[42m - Zöld háttér

# \033[43m - Sárga háttér

# \033[44m - Kék háttér

# \033[45m - Lila (Magenta) háttér

# \033[46m - Cián háttér

# \033[47m - Világos Szürke háttér

# \033[100m - Sötét Szürke háttér

# \033[101m - Világos Piros háttér

# \033[102m - Világos Zöld háttér

# \033[103m - Világos Sárga háttér

# \033[104m - Világos Kék háttér

# \033[105m - Világos Lila (Magenta) háttér

# \033[106m - Világos Cián háttér

# \033[107m - Fehér háttér

# Visszaállítás (mind színre, mind stílusra): \033[0m